# Semi-automatization of DRN and SFR sensitivity analysis/calibration

**Worflow**\
DRN
1. Load .drn file - *completed*
2. Change the conductance of the cells - *completed*
3. Write the .drn file - *completed*
4. Run the model - *in progress*
5. Read .cbb file
6. Store the outflow data of the first X cells
7. Put the process in a loop over a range of parameters

SFR
1. Load .sfr file - *in progress*
2. Change the hydraulic conductivity of the cells
3. Write the .sfr file
4. Run the model
5. Read streamflow.dat file
6. Store the flux data of cell at reach X
7. Put the process in a loop over a range of parameters

## Setup

In [1]:
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd

#check if the below lines are still useful at the end
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

In [2]:
# cwd = "C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/TEAM_Idrogeo/Tesi/Tesi_Ceola_Pirovano/Tesi_Pirovano/busca_drain_start"
cwd = os.getcwd()

In [3]:
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

## DRAIN

### Load .drn file

In [4]:
# Load .drn file
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn.head()

,layer,row,column,stage,conductance,node
0,1,46,51,124.838,0.012,0
1,1,47,51,124.835,0.018,0
2,1,48,51,124.832,0.006,0
3,1,48,52,124.830,0.024,0
4,1,49,52,124.824,0.018,0


In [5]:
# Load DRAIN cells characteristics
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))

In [14]:
# Compute the conductance of each cell for a given hydraulic conductivity
# Change the conductance column
k = 0.00001444444444444 #hydraulic conductivity
conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
drn.conductance = conductance_change
# drn

### Write .drn file using flopy

**bug found:** the layer gets put equal to 2 instead of 1
- the problem is not in the stress_period_data table passed to drn
- check stress_period_data.write_transient method
- couldn't figure it out, so found a way out: set drn.layer = 0
- drn_input_file_test_v4.drn works

In [8]:
modelpth = os.path.join(cwd, 'test_files')

# create the model class (only useful because it's needed by ModflowDrn class)
# this can be replaced by loading the actual model .nam file (not needed though)
mf = flopy.modflow.Modflow(
     "drn_test",
    model_ws = modelpth,
    exe_name = "mf2005",
)

In [9]:
# transform the drn structure in the flopy required format for "stress_period_data"
drn.layer = 0
stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

In [10]:
#generate the drn package inside the flopy class
ipakcb = 50 #code for cell-by-cell flow data storage
drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd, 'test_files', 'drn_input_file_test_v5.drn'))
drain.write_file(check = False)

### Test: Run the model

In [ ]:
# trial: run from flopy

# load the existing model busca_drain
# https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mf.html

mf = flopy.modflow.Modflow.load(
        os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.nam'),
        model_ws = os.path.join(cwd, 'test_files', 'drain_model_test'),
        exe_name='MF2005',
        version = 'mf2005',
        verbose = False
        )

In [ ]:
# drn stress period data can be accessed by:
mf.drn.stress_period_data.data

In [ ]:
# this works
success, buff = flopy.mbase.run_model(
                exe_name = os.path.join(cwd, 'test_files', 'drain_model_test','MF2005.exe'),
                namefile = 'busca_drain.nam',
                model_ws = os.path.join(cwd, 'test_files', 'drain_model_test')
                )
if not success:
    raise Exception("MODFLOW did not terminate normally.")

### Read .cbb output file

class CellBudgetFile\
https://flopy.readthedocs.io/en/latest/source/flopy.utils.binaryfile.html

The outflow will have to be considered until cell:
- row 92, column 76 (last cell of first segment)
Check if any substantial change in outflow happen if considering:
- row 93, column 76 (first cell of second segment)

In [ ]:
#get drain values from cbb output
cbb = bf.CellBudgetFile(os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.cbb'))
drain = cbb.get_data(text = 'DRAINS')

# get all the cells until 92, 76 & sum the flux extracted from drain[0]
row_limit = 92
column_limit = 76
sum_drain = np.sum(drain[0][0, :row_limit+1, :column_limit+1])
sum_drain

### LOOP

In [ ]:
#Setup
#Import all necessary packages
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

#Define working directory 
cwd = os.getcwd()   #where test files are stored (github)
# cwd_model = "d:/Claudia/MAURICE/test_models/"  #where model files are stored (local)

# Function to load .drn file directly
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

# Load original .drn file, load DRAIN cells characteristics and
# transform the drn structure in flopy (required format for "stress_period_data")
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))
drn.layer = 0

# Create and load the model class (only useful because it's needed by ModflowDrn class)
# this can be replaced by loading the actual model .nam file (not needed though)
modelpth = os.path.join(cwd, 'test_files', 'drain_model_test') #path to model
# mf = flopy.modflow.Modflow(
#     "busca_drain",
#     model_ws = os.path.join(cwd_model, 'drain_model_test'),
#     exe_name = "mf2005",
# )
mf = flopy.modflow.Modflow.load(
    os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.nam'),
    model_ws = os.path.join(cwd, 'test_files', 'drain_model_test'),
    exe_name='MF2005',
    version = 'mf2005',
    verbose = False
    )
'''
START OF LOOP
'''
# Define limits of hydraulic conductivity range and number of iterations
ki = 0.01
kf = 0.000001
n = 10 #10 to test the code, then switch to 100
step = np.linspace(ki,kf,n)
inputs = []
outputs = []

ipakcb = 50 #code for cell-by-cell flow data storage

# Loop to change conductance in the drain package, run the model and extract cbb results
for i in range(n):
    k = step[i]
    #change conductance
    conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
    drn.conductance = conductance_change
    stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

    #generate the drn package inside the flopy class
    drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.drn'))
    drain.write_file(check = False)
    
    #run the model
    success, buff = mf.run_model(silent=False) #False to test the code, then switch to True
    if not success:
        raise Exception("MODFLOW did not terminate normally.")
    
    #get drain values from cbb output
    cbb = bf.CellBudgetFile(os.path.join(cwd, 'test_files', 'drain_model_test', 'busca_drain.cbb'))
    drain = cbb.get_data(text = 'DRAINS')

    # get all the cells until 92, 76 & sum the flux extracted from drain[0]
    row_limit = 92
    column_limit = 76
    sum_drain = np.sum(drain[0][0, :row_limit+1, :column_limit+1])

    # Append the input and output to lists
    inputs.append(k)
    outputs.append(sum_drain)

    #print(k)

#save to dataframe and export
df_results = pd.DataFrame({'k_value': inputs, 'drain_results': outputs})
df_results.to_excel(os.path.join(cwd, 'test_files','results.xlsx'))

## SFR

### Test: load .sfr file

The .sfr file is multi structured:
- the above section has a structure similar to .drn
- in the last section, the structure is different: there is the info about the segments
- width and manning coefficient are defined in the segment section
- length, slope, bed thickness and bed hydraulic conductivity are defined in the first section
try to use flopy to load the model and then modify the sfr parameters directly from there

I had issues in loading the .sfr file through **ModflowSfr2.load()**\
- the file was structured as SFR2 input file, and the function didn't recognize it
- so we changed the input file structure to SFR1 from GWV
- now the file is correctly loaded

Once the file is loaded through ModflowSfr2.load(), it contains, among other things:
- **sfr.reach_data**: access the characteristics of the reaches
- **sfr.segment_data**: access the characteristics of the segments

In [46]:
# Load SFR package
m = flopy.modflow.Modflow()
f = os.path.join(cwd, 'test_files', 'sfr_model_test', 'test_sfr1_input', 'busca_sfr2_tp_start.sfr')

sfr = flopy.modflow.ModflowSfr2.load(f, m, gwt =True)

In [47]:
# Modify k
# In this configuration: equal k to all segments. We could also provide different k to the segments
k = 0.005
sfr.segment_data[0].hcond1 = [k for i in range(7)]
sfr.segment_data[0].hcond2 = [k for i in range(7)]

In [15]:
# Write the file
sfr.write_file(os.path.join(cwd, 'test_files', 'sfr_model_test', 'test_sfr1_input', 'busca_sfr2_tp.sfr'))

slope is 0!!!!\
it doesn't get saved into the .sfr file if SFR input file is set equal to SFR1 (NRSTM = 0)\
so, try to load the file in the other format\
try to load .sfr manually, get the parameters needed, then save them

In [17]:
# set isfropt to 1
m = flopy.modflow.Modflow()
f = os.path.join(cwd, 'test_files', 'sfr_model_test', 'busca_sfr2.sfr')

sfr = flopy.modflow.ModflowSfr2(m, isfropt = 1)

# sfr = flopy.modflow.ModflowSfr2.load(f, m, gwt =True)

In [19]:
# sfr.load(f, m, nper = 50)

#check file format
#be sure of the parameters set to sfr in gwv

#export as txt or shape
#import in flopy
#create the sfr package from the sfr2 class

# clean the script

In [10]:
xls = pd.ExcelFile(os.path.join(cwd, 'test_files', 'busca_sfr2_sfr_data.xlsx'))
items = xls.sheet_names
items

['ITEM1', 'ITEM2', 'ITEM5', 'ITEM6a', 'ITEM6b', 'ITEM6c']

In [91]:
# load reach data (item 2)
reach_data = pd.read_excel(os.path.join(cwd, 'test_files', 'busca_sfr2_sfr_data.xlsx'), sheet_name = 'ITEM2')
reach_data = reach_data.apply(pd.to_numeric)
reach_data.columns = [x.lower() for x in reach_data.columns]

reach_data.columns = ['k', 'i', 'j', 'iseg', 'ireach', 'rchlen', 'strtop', 'slope',  'strthick',  'strhc1']

# load segment data (item 6a)
segment_data = pd.read_excel(os.path.join(cwd, 'test_files', 'busca_sfr2_sfr_data_v2.xlsx'), sheet_name = 'ITEM6abc')
segment_data.columns = [x.lower() for x in segment_data.columns]

reach_data = reach_data.loc[:,:].to_records(index = False)
segment_data = segment_data.loc[:,:].to_records(index = False)
segment_data = {0: segment_data}

In [94]:
m = flopy.modflow.Modflow()

In [69]:
pd.read_excel(os.path.join(cwd, 'test_files', 'busca_sfr2_sfr_data.xlsx'), sheet_name = 'ITEM1')

,NSTRM,NSS,NSFRPAR,NPARSEG,CONST,DLEAK,ISTCB1,ISTCB2,ISFROPT,NOTSURE1,NOTSURE2,NOTSURE3
0,-328,7,0,0,1,0.0001,50,54,1,10,1,30


In [ ]:
nstrm = -len(reach_data)  # number of reaches
nss = len(segment_data[0])  # number of segments
nsfrpar = 0  # number of parameters (not supported)
nparseg = 0
const = 1  # constant for manning's equation: 1 for m/s
dleak = 0.0001  # closure tolerance for stream stage computation
ipakcb = 50  # flag for writing SFR output to cell-by-cell budget (on unit 53)
istcb2 = 54  # flag for writing SFR output to text file
dataset_5 = {0: [nss, 0, 0, 0]}  # dataset 5 (see online guide)

sfr = flopy.modflow.ModflowSfr2(
    m,
    nstrm=nstrm,
    nss=nss,
    const=const,
    dleak=dleak,
    ipakcb=ipakcb,
    istcb2=istcb2,
    reach_data=reach_data,
    segment_data=segment_data,
    dataset_5=dataset_5,
    unit_number=15,
    isfropt = 1
)

sfr.write_file(os.path.join(cwd, 'sfr_test_input.sfr'))

In [ ]:
# https://flopy.readthedocs.io/en/stable/Notebooks/mf6_sfr_tutorial01.html

In [89]:
test = np.genfromtxt(os.path.join(cwd, 'test_files', 'test1ss_segment_data.csv'), delimiter=',', names=True)


In [90]:
test

array([(1., 4., 2., 0., 11.,  25., 0.   , 0.   , 0.        , 0.        , 0.        , 0.        , 2.99999992e-05, 3., 1095.,  0., 0., 2.99999992e-05, 3., 1075.,  0., 0.),
       (2., 1., 6., 0.,  0.,   0., 0.03 , 0.   , 0.        , 0.        , 0.        , 0.        , 2.99999992e-05, 3., 1075., 12., 0., 2.99999992e-05, 3., 1050., 12., 0.),
       (3., 0., 5., 1.,  0.,  10., 0.   , 0.   , 0.        , 0.        , 0.        , 0.        , 2.99999992e-05, 2., 1075., 10., 2., 2.99999992e-05, 2., 1060.,  6., 1.),
       (4., 1., 5., 0.,  0.,  10., 0.03 , 0.   , 0.        , 0.        , 0.        , 0.        , 2.99999992e-05, 3., 1080., 10., 0., 2.99999992e-05, 3., 1060., 10., 0.),
       (5., 3., 6., 0.,  0.,   0., 0.   , 0.   , 0.30000001, 0.34999999, 3.79999995, 0.60000002, 2.99999992e-05, 3., 1060.,  0., 0., 2.99999992e-05, 3., 1045.,  0., 0.),
       (6., 1., 8., 0.,  0.,   0., 0.03 , 0.   , 0.        , 0.        , 0.        , 0.        , 2.99999992e-05, 3., 1045., 12., 0., 2.99999992e-05, 3

### Run the model

In [16]:
success, buff = flopy.mbase.run_model(
                exe_name = os.path.join(cwd, 'test_files', 'sfr_model_test', 'test_sfr1_input', 'MF2005.exe'),
                namefile = 'busca_sfr2_tp.nam',
                model_ws = os.path.join(cwd, 'test_files', 'sfr_model_test', 'test_sfr1_input')
                )
if not success:
    raise Exception("MODFLOW did not terminate normally.")

FloPy is using the following executable to run the model: MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_sfr2_tp.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/11/06  9:00:33

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/11/06  9:00:34
 Elapsed run time:  1.406 Seconds

  Normal termination of simulation


### Store the flow at reach x

Consider the outward flow of these two cells:
- reach 72, segment 1
- reach 1, segment 2

In [ ]:
# load streamflow.dat


## Other tests

### Test: write .drn file autonomously without using flopy

Riprovare con il pacchetto di flopy usando un numpy array invecedel dataframe


Da qui stavo scrivendo write_file per scrivermi i dati
https://flopy.readthedocs.io/en/latest/_modules/flopy/modflow/mfdrn.html#ModflowDrn.write_file

stavo indagando come flopy gestisce lo stress_period_data (MFils o qualcosa)

In [ ]:
ipakcb = 1
line = f"{2:10d}{ipakcb:10d}"
line

In [ ]:
heading = '# DRN package for MODFLOW-2005 generated by paolchol'
n_drain_cells = drn.shape[0]
ipakcb = 50


with open(os.path.join(cwd, 'drn_input_file_test.drn'), 'w') as file:
    file.write(f"{heading}\n")
    line = f"{n_drain_cells:10d}{ipakcb:10d}"
    # for opt in self.options:
        # line += " " + str(opt)
    line += "\n"
    file.write(line)

    # for row in drn.iterrows():
    #     file.writelines(row)